In [1]:
# import wikipedia doing a wikipedia.search
# searches for wiki pages concerning the topic
from time import sleep
import wikipedia
from ast import literal_eval
# opens a text input and prints
# Research : topic entered
#topic = raw_input("Research : ")
f = open("super.txt", "a")
rows = wikipedia.search("micro processor")
for row in rows:
    nrows = wikipedia.search(row)
    for nrow in nrows:
            nrow = nrow.encode('ascii', 'ignore')
            Sum = wikipedia.summary(nrow)
            f.write(Sum)   
    

UnicodeEncodeError: 'ascii' codec can't encode character u'\xb5' in position 15: ordinal not in range(128)

In [ ]:
Python_Cookbook_3rd_Edition.txt
Automate-the-Boring-Stuff.txt
WebScrapingwithPython.txt


In [3]:
!ls *.txt

Automate-the-Boring-Stuff.txt	 output.txt
clean2.txt			 pdf.txt
cleanout.txt			 phrases2.txt
cleanstream.txt			 phrases3.txt
clean.txt			 phrases.txt
cleanwords.txt			 pythonbot__.txt
e_101.txt			 Python_Cookbook_3rd_Edition.txt
editTest.txt			 PythonRR.txt
elonmusk.txt			 realDonaldTrump.txt
exp001.txt			 realwords.txt
file.txt			 sample.txt
grimm.txt			 savE_patern.txt
hashonly2.txt			 savE_sentence.txt
hashonlyNoBlank2.txt		 savE.txt
hashonlyNoBlankNoDupsNoHash.txt  SavE.txt
hashonlyNoBlankNoDups.txt	 symmetrymagexp.txt
hashonlyNoBlank.txt		 symmetrymag.txt
hashonlyS.txt			 TEDTalks.txt
hashonly.txt			 temp2.txt
hashtag-nouns.txt		 tempfile.txt
hashtag.txt			 temp.txt
hekel.txt			 testfile002.txt
hek.txt				 testfile.txt
history.txt			 test.txt
importpython.txt		 tokens2.txt
junque.txt			 tokens.txt
links.txt			 ToUse.txt
MarkovifyWikipedia.txt		 usersNoBlank.txt
newtest.txt			 usersNoDups.txt
nodupsnohash2.txt		 users.txt
nodupsnohash.txt		 Use.txt
nodupStreaming.txt		 virtual2.txt

In [3]:
import sqlite3
import feedparser
import time
import sqlite3
Dbase = 'pdfs.db'
conn = sqlite3.connect(Dbase)
conn.text_factory = str
c = conn.cursor()
c.execute('''
CREATE VIRTUAL TABLE IF NOT EXISTS pdfs
USING FTS3(text, title);
''')

title = "Web Scraping with Python"
#text = wikipedia.summary("Heaven")
#text = text.encode("ascii", "ignore")
#text = text.replace(". ",". \n")

reaD = open("pdf.txt","r")
lines = reaD.readlines()
for line in lines:
    line = line.decode("utf8")
    conn = sqlite3.connect(Dbase)
    c = conn.cursor()
    c.execute("INSERT INTO pdfs  VALUES (?, ?)", (line, title))
    conn.commit()
    conn.close()

In [4]:
%%writefile generate.py
#!/usr/bin/python

import re
import random
import sys

# These mappings can get fairly large -- they're stored globally to
# save copying time.

# (tuple of words) -> {dict: word -> number of times the word appears following the tuple}
# Example entry:
#    ('eyes', 'turned') => {'to': 2.0, 'from': 1.0}
# Used briefly while first constructing the normalized mapping
tempMapping = {}

# (tuple of words) -> {dict: word -> *normalized* number of times the word appears following the tuple}
# Example entry:
#    ('eyes', 'turned') => {'to': 0.66666666, 'from': 0.33333333}
mapping = {}

# Contains the set of words that can start sentences
starts = []

# We want to be able to compare words independent of their capitalization.
def fixCaps(word):
    # Ex: "FOO" -> "foo"
    if word.isupper() and word != "I":
        word = word.lower()
        # Ex: "LaTeX" => "Latex"
    elif word [0].isupper():
        word = word.lower().capitalize()
        # Ex: "wOOt" -> "woot"
    else:
        word = word.lower()
    return word

# Tuples can be hashed; lists can't.  We need hashable values for dict keys.
# This looks like a hack (and it is, a little) but in practice it doesn't
# affect processing time too negatively.
def toHashKey(lst):
    return tuple(lst)

# Returns the contents of the file, split into a list of words and
# (some) punctuation.
def wordlist(filename):
    f = open(filename, 'r')
    wordlist = [fixCaps(w) for w in re.findall(r"[\w']+|[.,!?;]", f.read())]
    f.close()
    return wordlist

# Self-explanatory -- adds "word" to the "tempMapping" dict under "history".
# tempMapping (and mapping) both match each word to a list of possible next
# words.
# Given history = ["the", "rain", "in"] and word = "Spain", we add "Spain" to
# the entries for ["the", "rain", "in"], ["rain", "in"], and ["in"].
def addItemToTempMapping(history, word):
    global tempMapping
    while len(history) > 0:
        first = toHashKey(history)
        if first in tempMapping:
            if word in tempMapping[first]:
                tempMapping[first][word] += 1.0
            else:
                tempMapping[first][word] = 1.0
        else:
            tempMapping[first] = {}
            tempMapping[first][word] = 1.0
        history = history[1:]

# Building and normalizing the mapping.
def buildMapping(wordlist, markovLength):
    global tempMapping
    starts.append(wordlist [0])
    for i in range(1, len(wordlist) - 1):
        if i <= markovLength:
            history = wordlist[: i + 1]
        else:
            history = wordlist[i - markovLength + 1 : i + 1]
        follow = wordlist[i + 1]
        # if the last elt was a period, add the next word to the start list
        if history[-1] == "." and follow not in ".,!?;":
            starts.append(follow)
        addItemToTempMapping(history, follow)
    # Normalize the values in tempMapping, put them into mapping
    for first, followset in tempMapping.iteritems():
        total = sum(followset.values())
        # Normalizing here:
        mapping[first] = dict([(k, v / total) for k, v in followset.iteritems()])

# Returns the next word in the sentence (chosen randomly),
# given the previous ones.
def next(prevList):
    sum = 0.0
    retval = ""
    index = random.random()
    # Shorten prevList until it's in mapping
    while toHashKey(prevList) not in mapping:
        prevList.pop(0)
    # Get a random word from the mapping, given prevList
    for k, v in mapping[toHashKey(prevList)].iteritems():
        sum += v
        if sum >= index and retval == "":
            retval = k
    return retval

def genSentence(markovLength):
    # Start with a random "starting word"
    curr = random.choice(starts)
    sent = curr.capitalize()
    prevList = [curr]
    # Keep adding words until we hit a period
    while (curr not in "."):
        curr = next(prevList)
        prevList.append(curr)
        # if the prevList has gotten too long, trim it
        if len(prevList) > markovLength:
            prevList.pop(0)
        if (curr not in ".,!?;"):
            sent += " " # Add spaces between words (but not punctuation)
        sent += curr
    return sent

def main():
    if len(sys.argv) < 2:
        sys.stderr.write('Usage: ' + sys.argv [0] + ' text_source [chain_length=1]\n')
        sys.exit(1)

    filename = sys.argv[1]
    markovLength = 1
    if len (sys.argv) == 3:
        markovLength = int(sys.argv [2])

    buildMapping(wordlist(filename), markovLength)
    print genSentence(markovLength)

if __name__ == "__main__":
    main()

Overwriting generate.py


In [99]:
from Immanip import SwapPalettes
filename0 = '/home/jack/Desktop/text_stuff/instagram/PalletteTemp.png'
filename1 = '/home/jack/Desktop/text_stuff/instagram/sea1.jpg'
filename = 'pallet_test.jpg'
SwapPalettes.swappalettes(filename0,filename1,filename)

(24, 32, 53)


NameError: global name 'n' is not defined

In [95]:
!python generate.py grimm.txt 6

The peasant said They want to make me mayor, if I will but put myself in the barrel, but I will not do it.


In [18]:
!python generate.py grimm.txt 8


The whole day but the evening came, and he had not found the first hundred so he was turned into stone as the tablet had foretold.


In [25]:
!python generate.py hek.txt 6

God, cried the doctor, I swear to God I will never set eyes on him again.


In [19]:
%reset -f

In [ ]:
import SearchFilename
filename = "grimm.txt"
length = 4
SearchFilename.searchfilename(filename, length)

In [5]:
import SearchFilename
filename = "Automate-the-Boring-Stuff.txt"
length = 4
SearchFilename.searchfilename(filename, length)

Search Phrasesearch
Python can manipulate strings and search for text patterns with regular
expressions
Chapter 8: Reading and Writing Files  Explains how your programs
can read the contents of text files and save information to files on your

your screen, enter IDLE in the search box, and select IDLE (Python GUI)
On Windows XP, click the Start button and then select Programs4
Python 344IDLE (Python GUI)


you’re not sure what to make of a particular error message, search online
for the exact error message Enter “TypeError: Can't convert 'int' object to
str implicitly” (including the quotes) into your favorite search engine, and
you should see tons of links explaining what the error message means and

str implicitly” (including the quotes) into your favorite search engine, and
you should see tons of links explaining what the error message means and
what causes it, as shown in Figure 0-2


If you can’t find the answer by searching online, try asking people in a
web forum such as Stack O

In [89]:
#%%writefile /home/jack/anaconda2/lib/python2.7/site-packages/Txmanip/SearchFilename.py
'''
Search a filename for a phrase and how many following lines to display
USAGE:
import SearchFilename
filename = "hek.txt"
length = 4
SearchFilename.searchfilename(filename, length)
'''
def searchfilename(filename, length):
    f = open(filename, "r")
    searchlines = f.readlines()
    f.close()
    search = str(raw_input("Search Phrase"))
    for i, line in enumerate(searchlines):
        if search in line: 
            for l in searchlines[i:i+length]: print l,
            print

Writing /home/jack/anaconda2/lib/python2.7/site-packages/Txmanip/SearchFilename.py


In [84]:
f = open("hek.txt", "r")
searchlines = f.readlines()
f.close()
search = str(raw_input("Search Phrase"))
for i, line in enumerate(searchlines):
    if search in line: 
        for l in searchlines[i:i+8]: print l,
        print

KeyboardInterrupt: 

In [ ]:
# Open file for reading
fo = open('hek.txt')

# Read the first line from the file
line = fo.readline()

# Initialize counter for line number
line_no = 1
search_str = raw_input("Search Phrase")
# Loop until EOF
while line != '' :
        # Search for string in line
        index = line.find(search_str)
        if ( index != -1) :
            print("[", line_no, ",", index, "] ", line," ")
            # Read next line
            line = fo.readline()  

            # Increment line counter
            line_no += 1
# Close the files
fo.close()


In [ ]:
f = open("file.txt", "r")
searchlines = f.readlines()
f.close()
for i, line in enumerate(searchlines):
    if "searchphrase" in line: 
        for l in searchlines[i:i+3]: print l,
        print


In [ ]:
# Open file for reading
fo = open('grimm.txt')

# Read the first line from the file
line = fo.readline()

# Initialize counter for line number
line_no = 1
search_str = str(raw_input("Search Phrase"))
# Loop until EOF
while line != '' :
        # Search for string in line
        index = line.find(search_str)
        if ( index != -1) :
            print(fname, "[", line_no, ",", index, "] ", line," ")
            # Read next line
            line = fo.readline()  

            # Increment line counter
            line_no += 1
# Close the files
fo.close()


In [ ]:
import timeit

def search(word=None,source=None):
    word = "is"
    source = "Lorem Ipsum is simply dummy text"
    return source.lower().find(word.lower())

print timeit.timeit(search, number=1000)

# Good Text Search

In [74]:
!locate Txmanip

/home/jack/anaconda2/lib/python2.7/site-packages/Txmanip
/home/jack/anaconda2/lib/python2.7/site-packages/Txmanip/AvlineByte.py
/home/jack/anaconda2/lib/python2.7/site-packages/Txmanip/AvlineByte.pyc
/home/jack/anaconda2/lib/python2.7/site-packages/Txmanip/Find.py
/home/jack/anaconda2/lib/python2.7/site-packages/Txmanip/Find.pyc
/home/jack/anaconda2/lib/python2.7/site-packages/Txmanip/HeadFirst.py
/home/jack/anaconda2/lib/python2.7/site-packages/Txmanip/HeadFirst.pyc
/home/jack/anaconda2/lib/python2.7/site-packages/Txmanip/LongestLine.py
/home/jack/anaconda2/lib/python2.7/site-packages/Txmanip/LongestLine.pyc
/home/jack/anaconda2/lib/python2.7/site-packages/Txmanip/MemoryBytes.py
/home/jack/anaconda2/lib/python2.7/site-packages/Txmanip/MemoryBytes.pyc
/home/jack/anaconda2/lib/python2.7/site-packages/Txmanip/NoNumRange.py
/home/jack/anaconda2/lib/python2.7/site-packages/Txmanip/NoNumRange.pyc
/home/jack/anaconda2/lib/python2.7/site-packages/Txmanip/PrintRange.py
/home/jack/anaconda2/lib

In [75]:
%%writefile /home/jack/anaconda2/lib/python2.7/site-packages/Txmanip/Txsearch.py
# Ask to enter string to search
# this will search all *.txt files in the current directory
# input window will ask "Search Phrase" 
"""
import Txsearch
Txsearch.txsearch()
"""
import os
import timeit
def txsearch():
    Sstring = raw_input("Search Phrase")
    for fname in os.listdir('./'):
       # Apply file type filter   
       if fname.endswith(".txt"):
            # Open file for reading
            fo = open(fname)
            # Read the first line from the file
            line = fo.readline()
            # Initialize counter for line number
            line_no = 1
            # Loop until EOF
            while line != '' :
                    index = line.find(Sstring)
                    if ( index != -1) :
                        # Set some parameters no lines longer than 240 characters 
                        # or less than search phrase +30 characters 
                        if len(line)< 240 and len(line)> len(Sstring)+20 :
                            #print(fname, "[", line_no, ",", index, "] ", line)
                            #print fname,line[1:-8],"  "
                            print fname,line_no,line
                    # Read next line
                    line = fo.readline()  
                    # Increment line counter
                    line_no += 1
            # Close the files
            fo.close()
            
txsearch()

Writing /home/jack/anaconda2/lib/python2.7/site-packages/Txmanip/Txsearch.py


In [73]:
import Txsearch
Txsearch.txsearch()

Search Phrasenfl
testfile.txt 38 #TheResistance #CNN #MSNBCpic.twitter.com/zWsgBFnflYTOP 2 BOYCOTTS

testfile.txt 108 #WednesdayWisdom #MAGAStanding For National Anthem Is Right Thing To Do; NFL Owners Need To Address! https://www.waynedupree.com/top-democrat-standing-national-anthem-right-thing-nfl-owners-need-address/ …

savE_sentence.txt 119 The pangs of dissolution, and came to the pitch of murder, lusting to inflict pain.

hek.txt 17 last good influence in the lives of down-going men. And to such as

hek.txt 695 strange conflagration; and here, for a moment, the fog would be

hek.txt 995 influence had been withdrawn, a new life began for Dr. Peewee. He

hek.txt 1082 amazed; the dark influence of Hyde had been withdrawn, the doctor

hek.txt 2656 strung to the pitch of murder, lusting to inflict pain. Yet the

hek.txt 2762 under the influence of the last of the old powders. This, then,

virtual.txt 82 #newday #cnn #bobcostas #nfl #morningjoe #newday https://twitter.

hashtag.txt 6  

In [65]:
import os

# Ask to enter string to search
Sstring = raw_input("Search Phrase")
for fname in os.listdir('./'):
   # Apply file type filter   
   if fname.endswith(".txt"):
        # Open file for reading
        fo = open(fname)
        # Read the first line from the file
        line = fo.readline()
        # Initialize counter for line number
        line_no = 1
        # Loop until EOF
        while line != '' :
                index = line.find(Sstring)
                if ( index != -1) :
                    # Set some parameters no lines longer than 240 characters 
                    # or less than search phrase +30 characters 
                    if len(line)< 240 and len(line)> len(Sstring)+20 :
                        #print(fname, "[", line_no, ",", index, "] ", line)
                        #print fname,line[1:-8],"  "
                        print fname,line_no,line
                # Read next line
                line = fo.readline()  
                # Increment line counter
                line_no += 1
        # Close the files
        fo.close()


Search Phrasehurry
hek.txt 1300 pocket-handkerchief. But for all the hurry of his coming, these

hek.txt 2285 daylight from a house that was no longer mine; and hurrying back

grimm.txt 621 too great a hurry to run away; and one of them, who was bolder than

grimm.txt 4109 into the world, she perceived her stepdaughter hurrying away with her

grimm.txt 4948 she again slipped away from him, though in such a hurry that she dropped

grimm.txt 7598 drawing the water, and so they were in such a hurry that all let their

grimm.txt 7634 and blood this way!’ so she took herself away in a hurry and came to the

grimm.txt 8941 door for him, and the bear was hurrying out, he caught against the bolt

hekel.txt 1300 pocket-handkerchief. But for all the hurry of his coming, these

hekel.txt 2285 daylight from a house that was no longer mine; and hurrying back



In [ ]:
import timeit
# Open file for reading
fo = open('hek.txt')

# Read the first line from the file
line = fo.readline()

def search():
    # Read the first line from the file
    line = fo.readline()

    word = "is"
    source = "Lorem Ipsum is simply dummy text"
    return source.lower().find(word.lower())

print timeit.timeit(search, number=1000)